In [ ]:
%config Completer.use_jedi = False # To make auto-complete faster

#Reloads imported files automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../utils/')

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker
from matplotlib import colormaps as mplcmaps
import matplotlib.cm as cm

from matplotlib_param_funcs import set_matplotlib_params,reset_rcParams
set_matplotlib_params()

In [ ]:
import miscellaneous_functions as MF
import compute_variables as CV
import map_functions as mapf
import load_sim
import load_data

In [ ]:
#CHOOSE

x_variable = "l"
y_variable = "b"

vel_x_variable = 'r'
vel_y_variable = 'l'

In [ ]:
degree_symbol = "^\circ"

symbol_dict = mapf.get_kinematic_symbols_dict(x_variable=x_variable,
                                                         y_variable=y_variable,
                                                         vel_x_variable=vel_x_variable,
                                                         vel_y_variable=vel_y_variable)

units_dict = mapf.get_kinematic_units_dict(position_variables=x_variable+y_variable,
                                                     vel_x_variable=vel_x_variable,
                                                     vel_y_variable=vel_y_variable,
                                                     degree_symbol=degree_symbol)

pos_symbols_dict,pos_units_dict = mapf.get_position_symbols_and_units_dict(degree_symbol=r"$%s$"%degree_symbol)

# Load

In [ ]:
general_path = '/Users/luismi/Desktop/MRes_UCLan/'

In [ ]:
zabs = True
# zabs = False

R0 = 8.1

GSR = True
# GSR = False

## Sim

In [ ]:
sim_choice = "708main"
# sim_choice = "708mainDiff4"
# sim_choice = "708mainDiff5"

rot_angle = 27
axisymmetric = False
pos_scaling = 1.7

filename = load_sim.build_filename(choice=sim_choice,rot_angle=rot_angle,R0=R0,axisymmetric=axisymmetric,zabs=zabs,pos_factor=pos_scaling,GSR=GSR)

In [ ]:
load_chunk = False

if not load_chunk:
    np_path = general_path+f"data/{sim_choice}/numpy_arrays/"
        
    df0 = load_sim.load_simulation(path=np_path,filename=filename)
else:
    if sim_choice == "708main" and rot_angle == 27 and not axisymmetric and zabs and sim_scaling == 1.7:
        pickle_name = "df_bulge_zabs.pkl"
        df0 = pd.read_pickle("708main_simulation/"+pickle_name)

## Data

In [ ]:
obs_errors = True
# obs_errors = False

data_zabs = True
# data_zabs = False

In [ ]:
data_path = general_path+"data/Observational_data/"

data = load_data.load_and_process_data(data_path=data_path, error_bool=obs_errors, zabs=zabs, R0=R0, GSR=GSR)

# Compute

In [ ]:
# data_bool = True
data_bool = False

# frac_error = True if not data_bool else True
frac_error = True if not data_bool else False

In [ ]:
save_path = general_path + "graphs/other_plots/MC_distance/"
MF.create_dir(save_path)

save_path += "data/" if data_bool else "model/"
MF.create_dir(save_path)

print(save_path)

In [ ]:
if data_bool:
    cuts_dict = {"FeH":[-1,2], "l":[-2,2],"R":[0,5]}
else:
#     cuts_dict={"age":[0,4],"R":[0,5],"l":[-2,2],"b":[0,0.01]} # nuclear disc
#     cuts_dict={"age":[4,7],"R":[0,],"l":[-2,2],"b":[3.5,4]} # young pop
        cuts_dict={"age":[4,7],"l":[-2,2],"b":[3.5,4]} # young pop no R cut

df = MF.apply_cuts_to_df(data if data_bool else df0, cuts_dict=cuts_dict)
print(len(df),"stars")

In [ ]:
if frac_error:
    d_frac_error = 0.2
    
    df["d_error"] = df["d"]*d_frac_error
    
    print(f"Working with {d_frac_error} fractional distance error")
else:
    assert "d_error" in df, "If not working with frac error, then 'd_error' must be already exist (did you mean to set data_bool to True?)"
    print("Got the error from data")

In [ ]:
funcs_dict = {
    "correlation": CV.calculate_correlation,
    "anisotropy": CV.calculate_anisotropy,
}

In [ ]:
func_name = "correlation"
# func_name = "anisotropy"

func = funcs_dict[func_name]

Rmax_MC = 3.5
originally_within_Rmax = df["R"]<=Rmax_MC

df_within_lim = df[originally_within_Rmax]
print(f"Total: {len(df)}. Within R≤3.5 lim: {len(df_within_lim)}\n")

true_value = func(vx=df_within_lim["vr"],vy=df_within_lim["vl"])
print(f"{func_name} true value: {true_value}")

repeat = 5000

In [ ]:
std_MC, MC_values, within_Rmax = CV.get_std_MC(function=func,df=df,Rmax=Rmax_MC,repeat=repeat)

std_boot, boot_vals = CV.get_std_bootstrap(function=func,vx=df_within_lim["vr"].values,vy=df_within_lim["vl"].values,repeat=repeat)

print(f"Mean\t MC: {np.mean(MC_values)}. Boot: {np.mean(boot_vals)}")
print(f"Median\t MC: {np.median(MC_values)}. Boot: {np.median(boot_vals)}")
print(f"Std\t MC: {std_MC}. Boot: {std_boot}")

In [ ]:
# save_bool = True
save_bool = False

In [ ]:
plt.rcParams["font.size"] = 20

fig,ax=plt.subplots()
ax.hist(MC_values,bins=50,color="blue",alpha=0.7,label=fr"MC values ($R={repeat}$)")
ax.hist(boot_vals,bins=50,color="green",alpha=0.5, label=fr"Bootstrap values ($R={repeat}$)")
ax.axvline(true_value,color="red",label="True value")
ax.set_xlabel(symbol_dict[func_name]); ax.set_ylabel(r"$N$",rotation=0,labelpad=20)
ax.legend()

if True:
    filename = func_name
    
    if not frac_error:
        filename += "dataErr/"
    
    filename += f"_{d_frac_error}fracErr"
    
    filename += "_" + MF.extract_str_from_cuts_dict(cuts_dict)
    
    print(filename)
    
    if save_bool:
        plt.savefig(save_path+filename+".png", bbox_inches="tight")
        print("Saved in",save_path)
    plt.show()

In [ ]:
if True: # plot scatter inside/outside limit
    
    fig,ax=plt.subplots()
    
    colors = ["grey","red","limegreen"]
    elements_dic = {
        "Stayed inside/outside": [(originally_within_Rmax&within_Rmax)|(~originally_within_Rmax&~within_Rmax), 1, colors[0],1,"grey"],
        "Moved outside": [originally_within_Rmax&~within_Rmax, 10, colors[1],0.5,"k"],
        "Moved inside": [~originally_within_Rmax&within_Rmax, 10, colors[2],0.5,"k"]
    }

    y_var = "vr"; ylabel = r"$v_r$" + units_dict["mean_vx"]
#     y_var = "d"; ylabel = r"$d~$[kpc]"

    for k in elements_dic:
        condition,size,color,lw,edgecolor = elements_dic[k]

        ax.scatter(x=df[condition]["R"], y=df[condition][y_var], color=color, s=size, label=f"{k} ({sum(condition)})",lw=lw,edgecolor=edgecolor)

    ax.axvline(x=3.5,color="grey",lw=1,linestyle="--")
    ax.text(x=3.5+0.1,y=0.85*ax.get_ylim()[1], s=r"$R=3.5~$kpc",color="grey",size=15)

    ax.set_xlabel(r"$R$ [kpc]"); ax.set_ylabel(ylabel)

    leg = ax.legend()

    for text,c in zip(leg.get_texts(),colors):
        text.set_color(c)
             
if True: # filename and save
    filename = f"{y_var}-R"
    filename += f"_{d_frac_error}fracErr"
    filename += "_" + MF.extract_str_from_cuts_dict(cuts_dict)

    print(filename)
    
    if save_bool:
        plt.savefig(save_path+filename+".png", bbox_inches="tight")
        print("Saved in",save_path)
    
    plt.show()

In [ ]:
fig,ax=plt.subplots()

MC_d = np.random.normal(loc=df["d"],scale=df["d_error"]) # this is an example - in the std calculation this is calculated {repeat} number of times

ax.hist(MC_d,bins=100,label=r"MC $d$")
ax.hist(df["d"],bins=100,color="red",alpha=0.4,label=r"Original $d$")
ax.hist(MC_d-df["d"],bins=100,color="k",histtype="step",label="Difference")
ax.set_xlabel(r"$d$ [kpc]"); ax.set_ylabel(r"$N$",rotation=0,labelpad=20)
ax.legend()
plt.show()